In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os
from tools import wait_for_node_finished
from aiida.engine import calcfunction, workfunction, submit, run, run_get_node
from aiida.orm import load_code, load_node

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')

if True:
    code = get_or_create_code('alamode.alm_opt',
                              computer_local,
                              '/home/max/Documents/alamode/bin/alm')
    code_alm_opt = code
else:
    code_alm_opt = WorkflowFactory('alamode.alm_ALM_opt')


code = get_or_create_code('alamode.anphon',
                          computer_local,
                          '/home/max/Documents/alamode/bin/anphon')
code_anphon = code

code_alm_opt, code_anphon


In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


In [ ]:
from os.path import expanduser
from tools import NodeBank

# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [ ]:
g_cwd = nodebank.load("cwd")
#g_dispfile_folder = nodebank.load("dispfile_folder")
#g_displace_result = nodebank.load("displace_result")
g_superstructure = nodebank.load("superstructure")
g_standardizedstructure = nodebank.load("standardizedstructure")


In [ ]:
g_prefix = nodebank.load('prefix')
g_norder = nodebank.load(f"{g_prefix.value}_norder")
g_factor = nodebank.load('factor')
g_prefix,g_norder, g_factor


# alm opt 
```
Description:

	alm mode="opt"
	
	default input filename: alm_opt.in
	default output filename: alm_opt.out

Inputs:
             code:  required  Code             The `Code` to use for this job.
            dfset:  required  List             DFSET
           norder:  required  Int              1 (harmonic) or 2 (cubic)
        structure:  required  StructureData    structure of cyrstal.
           cutoff:  optional  Dict             distance cutoff
              cwd:  optional  Str              directory where results are saved.
           fc2xml:  optional  List             xml file for the cubic term
         metadata:  optional                   
             mode:  optional  Str              mode of alm='opt'
            param:  optional  Dict             optional parameters
Outputs:
  force_constants:  required  List             force constants.
     input_ANPHON:  required  List             ANPHON file
    remote_folder:  required  RemoteData       Input files necessary to run the process will be stored in this folder node ...
          results:  required  Dict             
        retrieved:  required  FolderData       Files that are retrieved by the daemon will be stored in this node. By defa ...
     remote_stash:  optional  RemoteStashData  Contents of the `stash.source_list` option are stored in this remote folder ...

```

In [ ]:
g_alamodeextract_future = nodebank.load(f'{g_prefix.value}_extract')


In [ ]:
g_DFSET = g_alamodeextract_future.outputs.dfset
g_DFSET


In [ ]:
g_superstructure

In [ ]:
codetype =  "calculation"
try:
    if issubclass(code_alm_opt, WorkChain):
        codetype = "workchain"
except TypeError:
    codetype = "calculation"
    

if codetype== "workchain":
    print("workchain")
    code = code_alm_opt

    inputs = {"structure": g_superstructure , # 　安定構造
            "cwd" : Str(os.path.join(g_cwd.value, f"{g_prefix.value}_opt")),
            "cutoff_radii": List(list=[[-1]]),
            "nbody": List(list=[[2]]),
            "norder":  g_norder,
            "dfset": g_DFSET}
    
    
    key = f'{g_prefix.value}_ALM_opt'
    g_almoptfuture = nodebank.load(key, raise_error=False)
    if g_almoptfuture is None:
        g_almoptfuture = submit(code_alm_opt, **inputs)
        print(g_almoptfuture)
        wait_for_node_finished(g_almoptfuture)
        nodebank.dump(key, g_almoptfuture)   

else:
    print("code")
    code = code_alm_opt

    builder = code.get_builder()
    builder.structure = g_superstructure  # 　安定構造
    builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_opt"))
    builder.norder = g_norder
    builder.dfset = g_DFSET
    
    key = f'{g_prefix.value}_opt'
    g_almoptfuture = nodebank.load_code_or_wait_for_node(key, builder)
    if False:
        g_almoptfuture = nodebank.load(key, raise_error=False)
        if g_almoptfuture is None:
            g_almoptfuture = submit(builder)
            print(g_almoptfuture)
            wait_for_node_finished(g_almoptfuture)
            nodebank.dump(key, g_almoptfuture)


In [ ]:
g_almoptfuture

In [ ]:
g_almoptfuture.outputs.results.attributes

In [ ]:
g_almoptfuture.outputs.input_ANPHON.list_object_names()

In [ ]:
nodebank.dump("anphonfile_xml", g_almoptfuture.outputs.input_ANPHON)


In [ ]:
g_almoptfuture.outputs.results

# phonon dispersion

In [ ]:
g_root_wd = nodebank.load("root_wd")
g_root_wd  # 実行root directory


In [ ]:
g_almprefix = "_".join([g_standardizedstructure.get_formula(),
                        "x".join(map(str, g_factor.get_array("factor").reshape(-1))),
                        g_prefix.value
                        ])
g_dispersionprefix = Str(g_almprefix)
print(g_dispersionprefix)

In [ ]:
g_property = Str(os.path.join(g_root_wd.value, "property")
                 )  # property directory
os.makedirs(g_property.value, exist_ok=True)
g_primstructure = nodebank.load("primstructure")  # primitive cell


# alamode.anphon
```
Description:

	No description available

Inputs:
           code:  required  Code             The `Code` to use for this job.
            cwd:  required  Str              directory where results are saved.
         fcsxml:  required  List             xml file
      structure:  required  StructureData    primitive structure.
     kappa_spec:  optional  Int              
       metadata:  optional                   
           mode:  optional  Str              anphon mode
         norder:  optional  Int              1 (harmonic) or 2 (cubic)
          param:  optional  Dict             additional parameters
   phonons_mode:  optional  Str              phonon mode
         prefix:  optional  Str              string added to filename.
          qmesh:  optional  List             phonon k-mesh
Outputs:
             kl:  required  ArrayData        
        kl_file:  required  SinglefileData   
        kl_spec:  required  ArrayData        
   kl_spec_file:  required  SinglefileData   
    phband_file:  required  SinglefileData   
     phdos_file:  required  SinglefileData   
  remote_folder:  required  RemoteData       Input files necessary to run the process will be stored in this folder node ...
    result_file:  required  SinglefileData   
        results:  required  Dict             
      retrieved:  required  FolderData       Files that are retrieved by the daemon will be stored in this node. By defa ...
    thermo_file:  required  SinglefileData   
   remote_stash:  optional  RemoteStashData  Contents of the `stash.source_list` option are stored in this remote folder ...

```

In [ ]:
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_dispersionprefix
builder.cwd = g_property
builder.fcsxml = g_almoptfuture.outputs.input_ANPHON
builder.mode = Str("phonons")
builder.phonons_mode = Str("band")

g_phbandfuture = nodebank.load('phband', raise_error=False)
if g_phbandfuture is None:
    g_phbandfuture = submit(builder)
    print(g_phbandfuture)
    wait_for_node_finished(g_phbandfuture, 5)
    if not g_phbandfuture.is_finished_ok:
        raise
    nodebank.dump('phband',g_phbandfuture)

In [ ]:
g_phbandfuture.is_finished_ok

In [ ]:
g_phbandfuture.outputs.phband_file

# phonon DOS

In [ ]:
# from alamode_aiida.alm_input import make_alm_uniform_kmesh
# g_qmesh_value = make_alm_uniform_kmesh(g_primstructure.get_ase(),kspacing=0.001)
g_qmesh = List(list=[10, 10, 10])  # qmeshの定義
g_qmesh.get_list()


In [ ]:
g_almprefix = "_".join([g_primstructure.get_formula(),
                        "x".join(map(str, g_factor.get_array("factor").reshape(-1))),
                        g_prefix.value,
                       "q"+"x".join(map(str, g_qmesh.get_list()))
                        ])
print(g_almprefix)

g_dosprefix = Str(g_almprefix)
g_dosprefix  # dos用の識別子


In [ ]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_dosprefix
builder.cwd = g_property
builder.fcsxml = g_almoptfuture.outputs.input_ANPHON
builder.mode = Str("phonons")
builder.phonons_mode = Str("dos")
builder.qmesh = g_qmesh

g_phdosfuture = nodebank.load('phdos', raise_error=False)
if g_phdosfuture is None:
    g_phdosfuture = submit(builder)
    print(g_phdosfuture)
    wait_for_node_finished(g_phdosfuture, 5)
    if not g_phdosfuture:
        raise
    nodebank.dump('phdos',g_phdosfuture)

# plot dispersion

```
Description:

	Phonon band workchain.
	
	band_filenames should support valid_type (SinglefileData, FolderData).

Inputs:
  band_filenames:  required  Str, List, SinglefileData  phonon band filenames
             cwd:  required  Str                        directory where results are saved.
          prefix:  required  Str                        string added to filenames.
    img_filename:  optional  Str                        image filename
        metadata:  optional                             
        unitname:  optional  Str                        unit of energy
Outputs:
        img_file:  required  SinglefileData        
```

In [ ]:
g_phbandfuture.outputs.results.attributes


In [ ]:
phbandworkchain = WorkflowFactory('alamode.phband_img')
g_band_filenames = g_phbandfuture.outputs.phband_file
inputs = {"cwd": g_property, "band_file": g_band_filenames, "prefix": g_dispersionprefix}
print(inputs)


In [ ]:
#phbandimgworkchain = run(phbandworkchain, **inputs)
#phbandimgworkchain

In [ ]:
phbandimgworkchain = submit(phbandworkchain, **inputs)
print(phbandimgworkchain)
wait_for_node_finished(phbandimgworkchain)
if not phbandimgworkchain.is_finished_ok:
    raise

In [ ]:
phbandimgworkchain.outputs.img_file.list_object_names()[0]

In [ ]:
from wand.image import Image as WImage
filename = os.path.join(g_property.value, phbandimgworkchain.outputs.img_file.list_object_names()[0])
img = WImage(filename=filename)
img

# plot DOS

```
Description:

	Phonon DOS workchain.
	
	dos_filenames should support valid_type (SinglefileData, FolderData).

Inputs:
            cwd:  required  Str                        directory where results are saved.
  dos_filenames:  required  Str, List, SinglefileData  phonon dos
         prefix:  required  Str                        string added to filenames
   img_filename:  optional  Str                        image filename
       metadata:  optional                             
       unitname:  optional  Str                        unit of energy
Outputs:
       img_file:  required  SinglefileData             image file
```


In [ ]:

phdosworkchain = WorkflowFactory('alamode.phdos_img')

inputs = {"cwd": g_property, "dos_file": g_phdosfuture.outputs.phdos_file,
          "prefix": g_dosprefix}
print(inputs)

g_dosfuture = submit(phdosworkchain, **inputs)
print(g_dosfuture)
wait_for_node_finished(g_dosfuture)
if not g_dosfuture.is_finished_ok:
    raise

In [ ]:
g_dosfuture.attributes

In [ ]:
from wand.image import Image as WImage
filename = os.path.join(g_property.value, g_dosfuture.outputs.img_file.list_object_names()[0])
img = WImage(filename=filename)
img

## free energies
```
Description:

	No description available

Inputs:
           cwd:  required  Str                  directory where files are saved.
        prefix:  required  Str                  string added to filenames
   thermo_file:  required  Str, SinglefileData  thermo file
  img_filename:  optional  Str                  image file template
      metadata:  optional                       
Outputs:
      img_file:  required  SinglefileData       image file
```

In [ ]:
FreeenergyImgWorkChain = WorkflowFactory("alamode.freeenergy_img")
g_inputs = {'cwd': g_property, 'prefix': g_dosprefix,
            'thermo_file': g_phdosfuture.outputs.thermo_file}


freeenergyimgfuture = submit(FreeenergyImgWorkChain, **g_inputs)
print(freeenergyimgfuture)
wait_for_node_finished(freeenergyimgfuture)
if not freeenergyimgfuture.is_finished_ok:

    raise

In [ ]:
from wand.image import Image as WImage
filename = os.path.join(g_property.value, freeenergyimgfuture.outputs.img_file.list_object_names()[0])
img = WImage(filename=filename)
img